# Requirements

In [ ]:
'''

REQUIREMENTS BEFORE RUNNING THIS NOTEBOOK:

1.  Split OSM building polygons at vertices --> called the "splitlines" layer.

2.  Prepare the dataset to identify street-facing segments, using the provided script in this package.

3.  Review the street-facing segments in GIS, making manual adjustments as required.

4.  In GIS, calculate the following geometry attributes, using meters, decimal degrees and
    the coordinate system WGS 1984 Web Mercator Auxiliary Sphere:
    "Center_Lon", "Center_Lat"  --> longitude and latitude of central point of each line segment.
    "Start_Lon", "Start_Lat"    --> these become tuple coordinates for corner_a or corner_b. From GIS,
                                it is unknown which if the start or end coordinates are the left or right
                                side of the image, so the script determines this and renames these as
                                corner 1 (left) and corner 2 (right).
    "End_Lon", "End_Lat"        --> same as above, but the other corner of the line segment.
    "LINE_BEARING"              --> line bearing of each line segment, from 0 to 360 degrees.
    "LINE_LENGTH"               --> the geodesic length of each line segment. While not used in
                                the functions in this notebook, line length is required for subsequent
                                scripts in this package, so one should calculate this in GIS prior to export.

    * Make sure to re-calculate all of the above if any line segments are merged during manual review.

    The following columns are required for the notebook and can be created in this notebook, or elsewhere.
    "osm_id_final"  -->   a unique identifier for each building segment, usually adding "_1", "_2", etc. to the osm_id.
    "image_id"      -->   can be the same as osm_id_final for initial extraction, but in future additional
                          suffixes may be added such as "_left" or "_right" if some extracted images need to be split
                          in case of multiple buildings represented by one polygon in OSM.

    "perp_angle"    -->   the angle from the GSV camera in the street to a street-facing building line segment.

    Once the above three columns are created, it is suggested to save/export the dataframe to allow consistent
    reuse of the osm_id_final and image_id columns in subsequent scripts.

5.  Filter the "splitlines" layer to include only street-facing segments, and export from GIS to Excel:
    --> this notebook uses the filename 'frontlines_for_extraction.xlsx'


* Recommended to use high-RAM runtime for this notebook

'''

# Install & import required libraries

In [ ]:
pip install google_streetview

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for google_streetview: filename=google_streetview-1.2.9-py3-none-any.whl size=9777 sha256=c0b3152759fd8980cb085b51c9ed583b0771fc3b92d389f4094ac6b3ab7fd20d
  Stored in directory: /root/.cache/pip/wheels/8f/55/d0/074e47d0e3fede14e60ddcd7b1a59681e1f1c3fd5b56cef79d
  Created wheel for kwconfig: filename=kwconfig-1.1.7-py3-none-any.whl size=4974 sha256=69eb86fda7469371d119cca913de19538885d0ed0a13aa471c00cea56e2ea173
  Stored in directory: /root/.cache/pip/wheels/5d/ae/f3/4f084ead544ae0187acf5ef586c5ee24ec92e1029b3383a8ac
Successfully built google_streetview kwconfig


In [1]:
# import google_streetview.api
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import os
import numpy as np
import pandas as pd
import math
import os
from datetime import datetime
import time
import cv2
import json
import pyproj
gd = pyproj.Geod(ellps='WGS84')
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', None)

# Load and format dataframe

In [ ]:
# Set path for project directory
PROJECT_DIR = # ADD PATH TO PROJECT DIRECTORY, e.g. '/content/drive/MyDrive/Project_Name'
print('directory contents: ', os.listdir(PROJECT_DIR))
os.chdir(PROJECT_DIR)

directory contents:  []


In [3]:
DATA_FOLDER = # ADD PATH TO DATA DIRECTORY, e.g. '/content/drive/MyDrive/Project_Name/data/'

In [8]:
# Load frontlines_df, the file from GIS containing all building polygons split into line segments
# and filtered to include only street-facing line segments.

frontlines_df = pd.read_excel(DATA_FOLDER+'frontlines_for_extraction.xlsx')
frontlines_df

,OBJECTID,osm_id,osm_id_final,Start_Lon,Start_Lat,End_Lon,End_Lat,Center_Lon,Center_Lat,LINE_LENGTH,LINE_BEARING,st_facing_manual
0,3,83471230,83471230_3,7.706208,45.096171,7.706241,45.096734,7.706225,45.096452,62.635242,2.411950,1
1,5,83471233,83471233_1,7.705838,45.097673,7.705801,45.097166,7.705819,45.097419,56.362096,182.904191,1
2,11,83471234,83471234_3,7.703627,45.097317,7.703663,45.097824,7.703645,45.097571,56.373605,2.911479,1
3,15,83471237,83471237_3,7.707488,45.095631,7.707522,45.096194,7.707505,45.095913,62.635197,2.412039,1
4,17,83471238,83471238_1,7.705814,45.096960,7.705768,45.096427,7.705791,45.096694,59.357366,183.508419,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2388,9757,1214584770,1214584770_2,7.697674,45.105997,7.697440,45.106057,7.697557,45.106027,19.549598,290.143764,1
2389,9762,1214584771,1214584771_3,7.698083,45.106000,7.698037,45.105940,7.698060,45.105970,7.622774,208.206956,1
2390,9763,1214584771,1214584771_4,7.698037,45.105940,7.697752,45.106041,7.697895,45.105990,25.095010,296.561084,1
2391,9768,1214584772,1214584772_1,7.698279,45.106267,7.698200,45.106145,7.698239,45.106206,14.912131,204.734596,1


In [ ]:
# # # ONLY USE THIS CELL IF UNIQUE IDENTIFIERS NOT YET CREATED ('osm_id_final' and 'image_id')

# # # Make unique identifier for each line segment in building.
# frontlines_df['osm_id_final'] = frontlines_df.groupby('osm_id').cumcount().add(1).astype(str)
# frontlines_df['osm_id_final'] = frontlines_df['osm_id'].astype(str) + '_' + frontlines_df['osm_id_final']

In [9]:
# The field 'image_id' can be set to equal 'osm_id_final', unless the user
# has another image_id naming scheme.
# There could be a difference between image_id and osm_id_final if extracted
# images eventually get split (e.g. "_left" or "_right") during image review stages.
frontlines_df['image_id'] = frontlines_df['osm_id_final']

# If not done already, add the perpendicular angle, i.e. from a GSV camera towards
# a street_facing front. In ArcGIS, this is consistently the bearing of the line plus 90 degrees.
frontlines_df['perp_angle'] = frontlines_df['LINE_BEARING'] + 90
frontlines_df['perp_angle'] = frontlines_df['perp_angle'].apply(lambda x: x - 360 if x > 360 else x)
frontlines_df

,OBJECTID,osm_id,osm_id_final,Start_Lon,Start_Lat,End_Lon,End_Lat,Center_Lon,Center_Lat,LINE_LENGTH,LINE_BEARING,st_facing_manual,image_id,perp_angle
0,3,83471230,83471230_3,7.706208,45.096171,7.706241,45.096734,7.706225,45.096452,62.635242,2.411950,1,83471230_3,92.411950
1,5,83471233,83471233_1,7.705838,45.097673,7.705801,45.097166,7.705819,45.097419,56.362096,182.904191,1,83471233_1,272.904191
2,11,83471234,83471234_3,7.703627,45.097317,7.703663,45.097824,7.703645,45.097571,56.373605,2.911479,1,83471234_3,92.911479
3,15,83471237,83471237_3,7.707488,45.095631,7.707522,45.096194,7.707505,45.095913,62.635197,2.412039,1,83471237_3,92.412039
4,17,83471238,83471238_1,7.705814,45.096960,7.705768,45.096427,7.705791,45.096694,59.357366,183.508419,1,83471238_1,273.508419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2388,9757,1214584770,1214584770_2,7.697674,45.105997,7.697440,45.106057,7.697557,45.106027,19.549598,290.143764,1,1214584770_2,20.143764
2389,9762,1214584771,1214584771_3,7.698083,45.106000,7.698037,45.105940,7.698060,45.105970,7.622774,208.206956,1,1214584771_3,298.206956
2390,9763,1214584771,1214584771_4,7.698037,45.105940,7.697752,45.106041,7.697895,45.105990,25.095010,296.561084,1,1214584771_4,26.561084
2391,9768,1214584772,1214584772_1,7.698279,45.106267,7.698200,45.106145,7.698239,45.106206,14.912131,204.734596,1,1214584772_1,294.734596


In [ ]:
# Once complete, save the unique identifiers, also import to GIS so that only these
# unique identifiers are used for the line segments.
export_dir = # directory to save dataframe
export_fname = # file name to save + .xlsx'
frontlines_df.to_excel(os.path.join(export_dir,export_fname))

# Set global variables

In [ ]:
# API key for GSV
KEY = # INSERT API KEY HERE

FOV = 120 # Field of view to use when capturing GSV images, default is 120 degrees.
RADIUS = 20 # Radius to search for GSV camera within given coordinates, default is 20 meters.
PITCH_LST = [90, 110] # List of pitches to use when capturing GSV images.
                      # 90 degrees is horizontal; 110 degrees is the default second pitch
                      # to capture upper view of buildings.
ROTATION_CORRECTION = 0 # Correction, in degrees, added to rotation from perpendicular angle with
                        # rotation_one_camera function.
                        # Only used if the user wants to capture slightly wider extents than calculated by script.
                        # Default is zero.

# The main folder is where constituent image tiles are saved,
# give it a suitable project name. The date is automatcially appended.
MAIN_NAME = # ADD A FOLDER NAME WHERE IMAGES ARE SAVED, e.g. based on project title
MAIN_FOLDER = datetime.today().strftime('%Y-%m-%d') + '_' + MAIN_NAME # Data automatically added
print('main folder: ', MAIN_FOLDER)

# The stitch folder is where final stitched images are saved,
# add a suitable name.
STITCH_NAME = 'Stitched'
STITCH_FOLDER = os.path.join(MAIN_FOLDER,STITCH_NAME)
print('stitch folder: ', STITCH_FOLDER)

# Define functions

In [ ]:
def parse_dataframe(row, dataframe):
  """
  Returns variables from dataframe for use in for loop, when running image extraction script

  Arguments:
  dataframe             --  pd.DataFrame; must contain columns "image_id", "Center_Lon", "Center_Lat", "Start_Lon",
                            "Start_Lat", "End_Lon", "End_Lat", and "LINE_BEARING".
  row                   --  int; row number in dataframe.

  Returns:
  image_id              --  string; unique image identifier.
  id_folder             --  string; folder name, where constituent images will be saved within MAIN_FOLDER.
  center_lon            --  float; the longitude coordinate of the central point of the street-facing line segment.
  center_lat            --  float; the latitude coordinate of the central point of the street-facing line segment.
  corner_a_tup          --  tuple; the starting points (longitude, latitude) of the street-facing line segment.
  corner_b_tup          --  tuple; the ending points (longitude, latitude) of the street-facing line segment.
  perpendicular_angle   --  float; angle that is perpendicular to the facade to be captured.
  """
  # extract paramaters from dataframe and set variables for each building id
  image_id = dataframe.at[row,"image_id"]
  id_folder = os.path.join(MAIN_FOLDER,str(image_id))

  center_lon = dataframe.at[row,"Center_Lon"]
  center_lat = dataframe.at[row,"Center_Lat"]
  # center_string = str(center_lat)+','+str(center_lon)

  corner_a_lon = dataframe.at[row,"Start_Lon"]
  corner_a_lat = dataframe.at[row,"Start_Lat"]
  corner_a_tup = (corner_a_lon, corner_a_lat)

  corner_b_lon = dataframe.at[row,"End_Lon"]
  corner_b_lat = dataframe.at[row,"End_Lat"]
  corner_b_tup = (corner_b_lon, corner_b_lat)

  line_bearing = dataframe.at[row,"LINE_BEARING"]
  perpendicular_angle = dataframe.at[row,"perp_angle"]

  return image_id, id_folder, center_lon, center_lat, corner_a_tup, corner_b_tup, perpendicular_angle


def calculate_ctr_offset(perpendicular_angle, center_lon, center_lat, offset_dist=6):
  """
  Function to calculate an offset from center of building facade, in direction that is
  opposite the perpendicular_angle. This is a rough estimate of the GSV camera location
  which helps avoid "wrong camera" errors when a facade is near two streets, i.e. a corner.

  Arguments:
  perpendicular_angle  --  float; angle that is perpendicular to the facade to be captured.
  center_lon           --  float; longitude coordinate of central point of street-facing line segment.
  center_lat           --  float; latitude coordinate of central point of street-facing line segment.
  offset_dist          --  float; distance to offset from center of building facade.

  Returns:
  offset_coords        --  string; coordinates of offset location.
  """
  offset_angle = perpendicular_angle - 180
  offset_lon, offset_lat, _ = gd.fwd(center_lon, center_lat, offset_angle, offset_dist)
  offset_coords = str(offset_lat)+','+str(offset_lon)
  return offset_coords


def get_center_camera(perpendicular_angle, fov, center_lon, center_lat, save_folder, radius=20, offset_dist=6):
  """
  Functions to call GSV API to get central camera location by capturing one initial image
  of the facade at the perpendicular angle, which also returns metadata containing the camera coordinates

  Arguments:
  perpendicular_angle   --  float; angle that is perpendicular to the facade to be captured.
  fov                   --  int; the field of view requested from the GSV API.
  center_lon            --  float; longitude coordinate of central point of street-facing line segment.
  center_lat            --  float; latitude coordinate of central point of street-facing line segment.
  save_folder           --  string; the folder where the image and metadata are to be saved.
  radius                --  int; the maximum distance, in meters, from the specified center_lon and center_lat
                            to look for the nearest GSV camera.
  offset_dist           --  float; distance, in meters, to offset from center of building facade.

  Returns:
  camera_lon            --  float; longitude coordinate of the nearest GSV camera.
  camera_lat            --  float; latitude coordinate of the nearest GSV camera.
  img0                  --  numpy array image; this is the orthogonal view of the facade, which will be used
                            in subsequent steps as the initial image to which all others will be stitched to.
  """
  # estimate camera coordinates
  estd_camera_coords = calculate_ctr_offset(perpendicular_angle, center_lon, center_lat, offset_dist)

  params = [{
         'size': '640x640', # max size is 640x640 pixels
         'location': estd_camera_coords,
         'fov' : fov,
         'heading': perpendicular_angle,
         'pitch': 0,
         'radius' : radius, # try looking for camera within 20 meters of target location
         'return_error_code' : True,
         'source' : 'outdoor',
         'key': KEY
          }]
  print(params)

  # Create a results object from GSV API
  results = google_streetview.api.results(params)
  # Download images to save_folder
  results.download_links(os.path.join(save_folder, 'initial_camera_0'))
  sub_folder = os.path.join(save_folder, 'initial_camera_0')

  # read image location if json file shows there was no error in retreiving image
  json_file = os.path.join(sub_folder, 'metadata.json')
  with open(json_file, 'r') as f:
    data = json.load(f)
    status = data[0]["status"]
    if status == "NOT_FOUND" or status == "ZERO_RESULTS":
      return False, False, False
    camera_pos_dict = data[0]["location"]
    f.close
  camera_pos = str(camera_pos_dict['lat']) + ',' + str(camera_pos_dict['lng'])
  camera_lon = camera_pos_dict['lng']
  camera_lat = camera_pos_dict['lat']

  # img0 is the orthogonal view of the building facade
  img0_path = save_folder + '/initial_camera_0/' +'gsv_0.jpg'
  img0 = cv2.imread(img0_path, cv2.IMREAD_COLOR)
  img0 = cv2.cvtColor(img0, cv2.COLOR_BGR2RGB)
  return camera_lon, camera_lat, img0

def calculate_point_d(point1, point2, point_c):
  """
  Finds the point falling on the line between building corner 1 and 2.
  The GSV camera (point C) forms a line with point D which is perpendicular to the line from corner 1 to 2.
  This is used within the rotation_one_camera function.

  Arguments:
  point1   --  numpy array; vector of [longitude, latitude] of building corner 1
  point2   --  numpy array; vector of [longitude, latitude] of building corner 2
  point_c   --  numpy array; vector of [longitude, latitude] of GSV camera

  Returns:
  point_d   --  numpy array; vector of [longitude, latitude] where the perpendicular angle from
                point_c meets line from corner 1 to 2.
  """
  # Calculate the vector from point1 to point2
  vector_p1_p2 = point2 - point1

  # Calculate the vector from point1 to point_c
  vector_p1_pc = point_c - point1

  # Calculate the projection of vector_p1_pc onto vector_p1_p2
  t = np.dot(vector_p1_pc, vector_p1_p2) / np.dot(vector_p1_p2, vector_p1_p2)

  # Calculate the coordinates of point_d
  point_d = point1 + t * vector_p1_p2

  return point_d

def rotation_one_camera(camera_lon, camera_lat, corner_a_lon, corner_a_lat,
                        corner_b_lon, corner_b_lat, perpendicular_angle, fov, correction_factor=0):
  """
  Calculates azimuth from central camera to corner points of buildng with PyProj library.
  If the full building is not within the current field of view, the function calculates
  the rotation required to capture additional images left/right to view the corners of the building.

  Arguments:
  camera_lon          --  float; longitude of the camera position from GSV, read in from metadata file.
  camera_lat          --  float; latitude of the camera position from GSV, read in from metadata file.
  corner_a_lon        --  float; the longitude of corner_a, read in from csv file and passed into a global variable.
  corner_a_lat        --  float; the latitude of corner_a, read in from csv file and passed into a global variable.
  corner_b_lon        --  float; the longitude of corner_b, read in from csv file and passed into a global variable.
  corner_b_lat        --  float; the latitude of corner_b, read in from csv file and passed into a global variable.
  perpendicular_angle --  float; angle that is perpendicular to the facade to be captured.
  correction_factor   --  a number of degrees to add to the calculated camera rotation, in the event of consistent
                          discrepancies in coordinates between OpenStreetMap (the source data) and GSV.

  Returns:
  capture_1           --  boolean; value indicating whether a new photograph should be captured towards corner_1,
                          which is defined within the function as the building corner closer to field of view 1 (fov_1).
  capture_2           --  boolean; value indicating whether a new photograph should be captured towards corner_2,
                          which is defined within the function as the building corner closer to field of view 2 (fov_2).
  heading_1           --  a heading in degrees to photograph corner_1.
  heading_2           --  a heading in degrees to photograph corner_2.
  """

  # Define the angles from the central camera defining the Field of View (fov)
  fov_1 = (perpendicular_angle - fov/2)
  fov_2 = (perpendicular_angle + fov/2)
  if fov_2 > 360:
    fov_2 -= 360

  # Input points as vectors
  camera_vec = np.array([camera_lon, camera_lat, 0])
  corner_a_vec = np.array([corner_a_lon, corner_a_lat, 0])
  corner_b_vec = np.array([corner_b_lon, corner_b_lat, 0])

  # Find the point on the line between corner 1 and 2 where a perpendicular line
  # from the camera falls. Find distance from this point to corners.
  perp_point_vec = calculate_point_d(corner_a_vec, corner_b_vec, camera_vec)

  # Calculate cross product to determine which of corner_a or _b is on the left
  # of the photograph --> lower cross-product value goes to the left side
  corner_a_cross = np.cross(corner_a_vec - camera_vec, perp_point_vec - camera_vec)[2]
  corner_b_cross = np.cross(corner_b_vec - camera_vec, perp_point_vec - camera_vec)[2]

  if corner_b_cross < corner_a_cross:
    corner_1_lon, corner_1_lat = corner_b_lon, corner_b_lat
    corner_2_lon, corner_2_lat = corner_a_lon, corner_a_lat
  else:
    corner_1_lon, corner_1_lat = corner_a_lon, corner_a_lat
    corner_2_lon, corner_2_lat = corner_b_lon, corner_b_lat

  fwd_azimuth_1, _, _ = gd.inv(camera_lon, camera_lat, corner_1_lon, corner_1_lat)
  fwd_azimuth_2, _, _ = gd.inv(camera_lon, camera_lat, corner_2_lon, corner_2_lat)

  # Adjust fwd_azimuth_1 and _2 to work for 0-360 degree plane with North at 0 degrees
  if fov_1 > 0 and fwd_azimuth_1 < 0:
    fwd_azimuth_1 += 360
  if fwd_azimuth_2 < 0:
    fwd_azimuth_2 += 360

  # Determine if the angle to corner_1 (i.e. fwd_azimuth_1) is within fov_1
  # If so, set capture_1 to True
  if fwd_azimuth_1 >= fov_1:
      capture_1 = False # i.e. don't need to capture a new photo toward this corner
  elif (fov_2-fov) < fwd_azimuth_1 <= fov_2:
      capture_1 = False
  else:
      capture_1 = True
      rotation_1 = - np.abs(fov_1 - fwd_azimuth_1)

  # If capture_1 is True, format the angles from 0 to 360 degrees and set the
  # heading angle to be captured.
  if capture_1 == True:
    if rotation_1 < -360:
      rotation_1 += 360
    rotation_1 -= correction_factor # increase rotation_1 by the correction_factor by rotating more left
    heading_1 = perpendicular_angle + rotation_1 # rotation_1 is a negative number, heading_1 will rotate left of perpendicular
    if heading_1 < 0: # GSV seems to work better when headings are inputted as positive
      heading_1 += 360
  # If capture_1 is False, a rotation toward heading_1 is not required
  else:
    heading_1 = False

  # Determine if the angle to corner_2 (i.e. fwd_azimuth_2) is within fov_2
  # If so, set capture_2 to True
  if fwd_azimuth_2 <= fov_2:
      capture_2 = False # i.e. don't need to capture a new photo toward this corner
  elif fov_1 < fwd_azimuth_2 <= (fov_1+fov):
      capture_2 = False
  else:
      capture_2 = True
      rotation_2 = np.abs(fwd_azimuth_2 - fov_2)

  # If capture_2 is True, format the angles from 0 to 360 degrees and set the
  # heading angle to be captured.
  if capture_2 == True:
    rotation_2 += correction_factor # increase rotation_2 by the correction_factor by rotating more right
    heading_2 = perpendicular_angle + rotation_2 #rotation_2 is a positive number, heading_2 will rotate right of perpendicular
    if heading_2 < 0:
      heading_2 += 360
    if heading_2 > 360:
      heading_2 -= 360
  # If capture_1 is False, a rotation toward heading_1 is not required
  else:
    heading_2 = False

  return heading_1, heading_2

def parse_headings(perpendicular_angle, heading_1, heading_2):
  """
  Function to parse heading_1 and heading_2 into a list to be sent to GSV API
  to capture additional images towards corner_1 and corner_2.

  Arguments:
  perpendicular_angle --  float; angle that is perpendicular to the facade to be captured.
  heading_1           --  float; a heading in degrees to capture image of corner_1.
  heading_2           --  float; a heading in degrees to capture image of corner_2.

  Returns:
  heading_lst         --  list; a list of headings to be sent to GSV API.
  """
  # parse headings from rotation_one_camera function to create heading_lst
  # to send to GSV API to request photos
  heading_lst = []
  if heading_1 != False and heading_2 != False:
    heading_lst = [perpendicular_angle,heading_1,heading_2]
  elif heading_1 == False and heading_2 == False:
    heading_lst = [perpendicular_angle]
  elif heading_1 == False:
    heading_lst = [perpendicular_angle,heading_2]
  elif heading_2 == False:
    heading_lst = [perpendicular_angle,heading_1]

  return heading_lst

def get_gsv_photos(ortho_img, save_folder, camera_lon, camera_lat, head_lst, pitch_lst, fov, key, radius=20):
  """
  Calls GSV API to capture and download photos according to desired location, heading, and pitch.

  Arguments:
  ortho_img   --  numpy array image; this is the orthogonal view of the facade returned as img0 in a previous function
                  used as the initial image to which all others will be stitched to.
  save_folder --  string; the folder where the images and metadata are to be saved.
  camera_lon  --  float; longitude coordinate of the camera position from GSV, read in from metadata file.
  camera_lat  --  float; latitude coordinate of the camera position from GSV, read in from metadata file.
  head_lst    --  list; list headings of images to be captured from the GSV camera, ranging 1 to 3 headings
                  per camera location, where head_lst[0] is always the orthogonal image at the perpendicular angle.
  pitch_lst   --  list; list of pitches (in degrees) to be captured from the GSV camera.
  fov         --  int; the field of view to be used when capturing GSV photos.
  key         --  string; the user's API key, issued by Google.
  radius      --  int; number of meters that GSV should search for one of its cameras from a target location.

  Returns:
  img_lst     --  list; list of the downloaded images, each of which is a numpy array image.
  """
  # initialize the img_lst with the orthogonal image at index [0]
  img_lst = [ortho_img]
  filepath_lst = []
  photo_counter = 1
  # set camera location with GSV required format
  gsv_cam_location = str(camera_lat)+','+str(camera_lon)

  # run loop to capture photos from one location to all headings in head_lst
  for i in range(len(head_lst)):
    heading = head_lst[i]

    # create for loop based on photo pitches in pitch_lst
    for j in range(len(pitch_lst)):
      if i == 0 and j == 0: # in this case, we already have the photo from the 1st API call
        continue
      # set parameters for photo
      params = [{
            'size': '640x640', # max size is 640x640 pixels
            'location': gsv_cam_location,
            'fov' : fov,
            'heading': heading,
            'pitch': (pitch_lst[j]-90),
            'radius' : radius,
            'return_error_code' : True,
            'source' : 'outdoor',
            'key': KEY
            }]

      # create a results object from GSV API and download images to directory
      results = google_streetview.api.results(params)
      results.download_links(os.path.join(save_folder, 'camera' + str(i) + '/shot_' + str(photo_counter)))
      sub_folder = os.path.join(save_folder, 'camera' + str(i) + '/shot_' + str(photo_counter))

      # read image if json file shows there was no error in retreiving image
      json_file = os.path.join(sub_folder, 'metadata.json')
      with open(json_file, 'r') as f:
        data = json.load(f)
        status = data[0]["status"]
        if status == "NOT_FOUND" or status == "ZERO_RESULTS":
          continue
        f.close

      # set file path, read images and save to image_lst
      filepath = os.path.join(sub_folder, 'gsv_0.jpg')
      filepath_lst.append(filepath)
      img = cv2.imread(filepath, cv2.IMREAD_COLOR)
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      img_lst.append(img)

      # increase photo_counter to have unique photo names
      photo_counter += 1

  return img_lst

def stitch_it(img_src, img_target):
  """
  Stitches two images together by computing homography, starting with the orthogonal image (img_src) at the bottom
  and warping perspective of the second image (img_target) on top of this. Uses SIFT as feature detector.
  This function is sed within stitch_two, stitch_four, and stitch_six functions, which add automatic column
  or row cropping in between rounds of stitching.

  Arguments:
  img_src       --  numpy array image, color; source image to be read into numpy array from .jpg outside of function.
                    This image must be orthogonal, i.e. taken at a heading perpendicular to the building, using a pitch of 90 degrees.
  img_target    --  numpy array image, color; target image to be read into numpy array from .jpg outside of function.
                    This image will be warped and stitched to the source image.

  Returns:
  stitched_img  --  numpy array image, color; the resulting stitched image
  len(matches)  --  number of keypoint matches used to find homography, can be saved in logs in subsequent functions.
  """

  #Convert images to grayscale to do feature detection
  img_src_gray = cv2.cvtColor(img_src, cv2.COLOR_BGR2GRAY)
  img_target_gray = cv2.cvtColor(img_target, cv2.COLOR_BGR2GRAY)

  # From: https://docs.opencv.org/3.4/d1/de0/tutorial_py_feature_homography.html
  MIN_MATCH_COUNT = 10

  # Detect SIFT features and compute descriptors
  MAX_NUM_FEATURES = 500
  sift = cv2.SIFT_create(MAX_NUM_FEATURES)
  keypoints_src, descriptors_src = sift.detectAndCompute(img_src_gray, None)
  keypoints_target, descriptors_target = sift.detectAndCompute(img_target_gray, None)

  # Find nearest match with KNN algorithm
  FLANN_INDEX_KDTREE = 1
  index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
  search_params = dict(checks=50)

  # Match features
  flann = cv2.FlannBasedMatcher(index_params, search_params)
  matches = flann.knnMatch(descriptors_src, descriptors_target, k=2)

  # Store all good matches as per Lowe's ratio test
  good = []
  for m, n in matches:
    if m.distance < 0.7*n.distance:
      good.append(m)
  matches = good
  print('no. good matches: ', len(matches))

  # FIND HOMOGRAPHY
  # Extract location of good matches
  points_src = np.zeros((len(matches),2), dtype=np.float32)
  points_target = np.zeros((len(matches),2), dtype=np.float32)

  for idx0, match in enumerate(matches):
    points_src[idx0, :] = keypoints_src[match.queryIdx].pt
    points_target[idx0, :] = keypoints_target[match.trainIdx].pt

  # Find homography
  H, mask = cv2.findHomography(points_target, points_src, cv2.RANSAC)

  # Use homography to warp image
  # Adapted from:
  # https://stackoverflow.com/questions/13063201/how-to-show-the-whole-image-when-using-opencv-warpperspective
  h0,w0 = img_src.shape[:2]
  h1,w1 = img_target.shape[:2]
  pts0 = np.float32([[0,0],[0,h0],[w0,h0],[w0,0]]).reshape(-1,1,2)
  pts1 = np.float32([[0,0],[0,h1],[w1,h1],[w1,0]]).reshape(-1,1,2)
  pts1_ = cv2.perspectiveTransform(pts1, H)
  pts = np.concatenate((pts0, pts1_), axis=0)
  [xmin, ymin] = np.int32(pts.min(axis=0).ravel() - 0.5)
  [xmax, ymax] = np.int32(pts.max(axis=0).ravel() + 0.5)
  t = [-xmin,-ymin]
  Ht = np.array([[1,0,t[0]],[0,1,t[1]],[0,0,1]]) # translate

  stitched_img = cv2.warpPerspective(img_target, Ht.dot(H), (xmax-xmin, ymax-ymin), cv2.INTER_LINEAR, cv2.BORDER_REPLICATE)
  stitched_img[t[1]:h0+t[1],t[0]:w0+t[0]] = img_src

  return stitched_img, len(matches)

def autocrop_col(img_src, crop_thresh=0.15):
  """
  Automatically crops images when more than a crop threshold (default = 15%) of the COLUMN is black.
  Used after FIRST round of image stitching.

  Arguments:
  img_src       --  numpy array image, color; source image to be cropped to where most of column
                    (i.e. greater than crop threshold) is black.
  crop_thresh   --  float; if more than this value (default = 15%) of the row is black, the column is cropped.


  Returns:
  img_crop      --  numpy array image, color; image cropped to where most of row (i.e. greater than threshold) is black.
  """
  # convert to gray and threshold image
  gray = cv2.cvtColor(img_src,cv2.COLOR_BGR2GRAY)
  _,thresh = cv2.threshold(gray,1,255,cv2.THRESH_BINARY)
  # the target is the value if all pixels in column times the crop threshold are white
  target = 255 * (1 - crop_thresh) * thresh.shape[0]
  # get the sum of values of each column, store in summmed array
  summed = np.sum(thresh, axis=0)
  # iterate through array, collect indexes where the value of a summed column is less than the target
  for idx1 in range(summed.shape[0]):
    if summed[idx1] < target:
      thresh[:,idx1] = 0
  # make bounding rectangle according to crop indexes and crop image
  x,y,w,h = cv2.boundingRect(thresh)
  img_crop = img_src[y:y+h,x:x+w]
  return img_crop

def autocrop_row(img_src, crop_thresh=0.15):
  """
  Automatically crops images when more than a crop threshold (default = 15%) of the ROW is black.
  Used after SECOND round of image stitching.

  Arguments:
  img_src       --  numpy array image, color; source image to be cropped to where most of row
                    (i.e. greater than crop threshold) is black.
  crop_thresh   --  float; if more than this value (default = 15%) of the row is black, the row is cropped.


  Returns:
  img_crop      --  numpy array image, color; image cropped to where most of row (i.e. greater than threshold) is black.
  """
  # convert to gray and threshold image
  gray = cv2.cvtColor(img_src,cv2.COLOR_BGR2GRAY)
  _,thresh = cv2.threshold(gray,1,255,cv2.THRESH_BINARY)
  # the target is the value if all pixels in row times the crop threshold are white
  target = 255 * (1 - crop_thresh) * thresh.shape[1]
  # get the sum of values of each row, store in summmed array
  summed = np.sum(thresh, axis=1)
  # iterate through array, collect indexes where the value of a summed row is less than the target
  for idx2 in range(summed.shape[0]):
    if summed[idx2] < target:
      thresh[idx2,:] = 0
  # make bounding rectangle according to crop indexes and crop image
  x,y,w,h = cv2.boundingRect(thresh)
  img_crop = img_src[y:y+h,x:x+w]
  return img_crop

def autocrop_sky(img_src, crop_thresh=0.98, sky_thresh=150):
  """
  Automatically crops sky from the top of images when percentage of the ROW
  (default = 98%) is white based on threshold of pixel value (default = 150).
  Used after FINAL round of image stitching.

  Arguments:
  img_src       --  numpy array image, color; source image where sky is to be cropped.
  crop_thresh   --  float; if more than this value (default = 98%) of the row is white, the row is cropped.
  sky_thresh    --  int; defines pixel brightness threshold; if pixel value is greater than this value
                    (default = 150), the row is cropped.

  Returns:
  img_crop      --  numpy array image, color; image cropped to where most of row (i.e. greater than threshold) is black.
  """
  # convert to gray and threshold image
  gray = cv2.cvtColor(img_src,cv2.COLOR_BGR2GRAY)
  _,thresh = cv2.threshold(gray,sky_thresh,255,cv2.THRESH_BINARY)
  # the target is the value if all pixels in row times the crop threshold are white
  target = 255 * crop_thresh * thresh.shape[1]
  summed = np.sum(thresh, axis=1)
  # iterate through array, collect indexes where the value of a summed row exceeds the target
  for idx3 in range(summed.shape[0]):
    if summed[idx3] > target:
      thresh[idx3,:] = 0
  # crop image according to indexes
  thresh[0:10,:] = 0 # the first rows are often non-zero due to black lines introduced during image stitching
  nonzero = np.nonzero(np.sum(thresh, axis = 1))[0]
  img_crop = img_src[nonzero[0]:nonzero[-1],:]
  return img_crop


def stitch_six(img_lst, show_images= False, save_file=False, save_directory=None, save_filename=None, write_logs=False):
  """
  Stitches six images together in three rounds, including automatic cropping of columns and/or rows
  (as required) in between rounds of stitching, with final crop of the sky.

  Given input images 0,1,2,3,4, and 5, there are 3 rounds of image stitching:
  First round images: 01, 23, 45
  Second round images: 0123, 0145
  Third round images: 012345 (i.e. all images)

  Arguments:
  img_lst         --  list; a list of 6 images to be stitched together,
                      each of which is a color numpy array inage.
  show_images     --  boolean, whether the images should be plotted using matplotlib upon completion.
  save_file       --  boolean, whether the files should be saved upon completion of the stitching.
  save_directory  --  string; location where the file should be saved to.
  save_filename   --  string; name of the file to be saved, e.g. 'stitch6.jpg'.
  write_logs      --  boolean; whether to write JSON file to log stitching keypoint matches for later analysis.

  Returns:
  final_crop      --  numpy array image, color; the stitched and cropped image result.
  """

  stitched_lst1 = [] # first round of stitched images to be stored in this list
  stitched_lst2 = [] # second round of stitched images to be stored in this list
  crop_lst1 = [] # first round of cropped images to be stored in this list
  crop_lst2 = [] # second round of cropped images to be stored in this list
  matches_lst = [] # list where number of keypoint matches will be saved
  i, j, k, l, m = 0, 0, 0, 0, 0 # reset indexes

  # Show original images
  if show_images == True:
    num_cols = 3
    num_rows = 2
    idx = 0
    plt.figure(figsize=[10,10])
    for idx in range(0, len(img_lst)):
      plt.subplot(num_rows, num_cols, idx + 1)
      plt.title('Original Images')
      plt.axis("off")
      plt.imshow(img_lst[idx])

  # First round of image stitching --> six images in img_lst: 0, 1, 2, 3, 4, and 5
  # Upon stitching, three resulting stitched images: 01, 23, and 45, stored in stitched_lst1
  for i in range(0,len(img_lst)-1,2):
    # call stitch_it function
    stiched_img, num_matches = stitch_it(img_lst[i], img_lst[i+1])
    matches_lst.append(num_matches)
    stitched_lst1.append(stiched_img)

  if show_images == True:
    num_cols = 3
    num_rows = 1
    idx = 0
    plt.figure(figsize=[10,10])
    for idx in range(0, len(stitched_lst1)):
      plt.subplot(num_rows, num_cols, idx + 1)
      plt.title('Stitch 1')
      plt.axis("off")
      plt.imshow(stitched_lst1[idx])

  # Crop black columns from first-round stitches
  for j in range(len(stitched_lst1)):
    crop = autocrop_col(stitched_lst1[j])
    crop_lst1.append(crop)

  if show_images == True:
    num_cols = 3
    num_rows = 1
    idx = 0
    plt.figure(figsize=[10,10])
    for idx in range(0, len(crop_lst1)):
      plt.subplot(num_rows, num_cols, idx + 1)
      plt.title('Stitch 1 Crop')
      plt.axis("off")
      plt.imshow(crop_lst1[idx])

  # Second round of image stitching --> three images in stitched_lst1: 01, 23, and 45
  # Upon stitching, two resulting stitched images: 0123, and 0145, stored in stitched_lst2
  for k in range(len(crop_lst1)-1):
    stitched_img2, num_matches = stitch_it(crop_lst1[0], crop_lst1[k+1])
    matches_lst.append(num_matches)
    stitched_lst2.append(stitched_img2)

  if show_images == True:
    num_cols = 2
    num_rows = 1
    idx = 0
    plt.figure(figsize=[10,10])
    for idx in range(0, len(stitched_lst2)):
      plt.subplot(num_rows, num_cols, idx + 1)
      plt.title('Stitch 2')
      plt.axis("off")
      plt.imshow(stitched_lst2[idx])

  # Crop black rows from second-round stitches
  for l in range(len(stitched_lst2)):
    crop2 = autocrop_row(stitched_lst2[l])
    crop_lst2.append(crop2)

  if show_images == True:
    num_cols = 2
    num_rows = 1
    idx = 0
    plt.figure(figsize=[10,10])
    for idx in range(0, len(crop_lst2)):
      plt.subplot(num_rows, num_cols, idx + 1)
      plt.title('Stitch 2 Crop')
      plt.axis("off")
      plt.imshow(crop_lst2[idx])

  # Third and final round of image stitching --> two images in stitched_lst2: 0123 and 0145
  # Upon stitching, only one stitched image: 012345, stored in stitched_img3
  for m in range(len(crop_lst2)-1):
    stitched_img3, num_matches = stitch_it(crop_lst2[0], crop_lst2[m+1])
    matches_lst.append(num_matches)

  if show_images == True:
    plt.figure(figsize=[10,10])
    plt.title('Final Stitch')
    plt.imshow(stitched_img3)

  # Crop sky and any remaining black columns/rows from third-round stitches
  final_crop = autocrop_col(stitched_img3)
  final_crop = autocrop_row(final_crop)
  final_crop = autocrop_sky(final_crop)

  if show_images == True:
    plt.figure(figsize=[10,10])
    plt.title('Final Crop')
    plt.imshow(final_crop)

  if save_file == True:
    # Write file to disk
    cv2.imwrite(os.path.join(save_directory, save_filename), cv2.cvtColor(final_crop, cv2.COLOR_BGR2RGB))

  if write_logs == True:
    # Log matches into JSON file for later analysis
    match_logpath = os.path.join(MAIN_FOLDER,'match_log.json')
    match_data = {}
    match_data['image_id'] = image_id
    match_data['matches'] = matches_lst
    with open(match_logpath, 'a') as f:
      f.write(json.dumps(match_data, indent=4))
    f.close

  return final_crop


def stitch_four(img_lst, show_images=False, save_file=False, save_directory=None, save_filename=None, write_logs=False):
  """
  Stitches four images together in two rounds, including automatic cropping of columns and/or rows
  (as required) in between rounds of stitching, with final crop of the sky.

  Given input images 0,1,2, and 3 there are 2 rounds of image stitching:
  First round images: 01, 23
  Second round images: 0123 (i.e. all images)

  Arguments:
  img_lst         --  list; a list of 6 images to be stitched together,
                      each of which is a color numpy array inage.
  show_images     --  boolean, whether the images should be plotted using matplotlib upon completion.
  save_file       --  boolean, whether the files should be saved upon completion of the stitching.
  save_directory  --  string; location where the file should be saved to.
  save_filename   --  string; name of the file to be saved, e.g. 'stitch6.jpg'.
  write_logs      --  boolean; whether to write JSON file to log stitching keypoint matches for later analysis.

  Returns:
  final_crop      --  numpy array image, color; the stitched and cropped image result.
  """
  stitched_lst1 = [] # first round of stitched images to be stored in this list
  crop_lst1 = [] # first round of cropped images to be stored in this list
  matches_lst = [] # list where number of keypoint matches will be saved
  i, j, k = 0, 0, 0 # reset indexes

  # Show original images
  if show_images == True:
    num_cols = 2
    num_rows = 2
    idx = 0
    plt.figure(figsize=[10,10])
    for idx in range(0, len(img_lst)):
      plt.subplot(num_rows, num_cols, idx + 1)
      plt.title('Original Images')
      plt.axis("off")
      plt.imshow(img_lst[idx])

  # First round of image stitching --> four images in img_lst: 0, 1, 2, and 3
  # Upon stitching, two resulting stitched images: 01 and 23, stored in stitched_lst1
  for i in range(0,len(img_lst)-1,2):
    stiched_img, num_matches = stitch_it(img_lst[i], img_lst[i+1])
    matches_lst.append(num_matches)
    stitched_lst1.append(stiched_img)

  if show_images == True:
    num_cols = 2
    num_rows = 1
    idx = 0
    plt.figure(figsize=[10,10])
    for idx in range(0, len(stitched_lst1)):
      plt.subplot(num_rows, num_cols, idx + 1)
      plt.title('Stitch 1')
      plt.axis("off")
      plt.imshow(stitched_lst1[idx])

  # Crop black columns from first-round stitches
  for j in range(len(stitched_lst1)):
    crop = autocrop_col(stitched_lst1[j])
    crop_lst1.append(crop)

  if show_images == True:
    num_cols = 3
    num_rows = 1
    idx = 0
    plt.figure(figsize=[10,10])
    for idx in range(0, len(crop_lst1)):
      plt.subplot(num_rows, num_cols, idx + 1)
      plt.title('Stitch 1 Crop')
      plt.axis("off")
      plt.imshow(crop_lst1[idx])

  # Second and final round of image stitching --> two images in img_lst: 01 and 23
  # Upon stitching, only one stitched image: 0123, stored in stitched_img2
  for k in range(len(crop_lst1)-1):
    stitched_img2, num_matches = stitch_it(crop_lst1[0], crop_lst1[k+1])
    matches_lst.append(num_matches)

  if show_images == True:
    plt.figure(figsize=[10,10])
    plt.title('Stitch 2')
    plt.axis("off")
    plt.imshow(stitched_img2)

  # Crop sky and any remaining black columns/rows from second-round stitches
  final_crop = autocrop_row(stitched_img2)
  final_crop = autocrop_col(final_crop)
  final_crop = autocrop_sky(final_crop)

  if show_images == True:
    plt.figure(figsize=[10,10])
    plt.title('Final Crop')
    plt.imshow(final_crop)

  if save_file == True:
    # Write file to disk
    cv2.imwrite(os.path.join(save_directory, save_filename), cv2.cvtColor(final_crop, cv2.COLOR_BGR2RGB))

  if write_logs == True:
    # Log matches into JSON file for later analysis
    match_logpath = os.path.join(MAIN_FOLDER,'match_log.json')
    match_data = {}
    match_data['image_id'] = image_id
    match_data['matches'] = matches_lst
    with open(match_logpath, 'a') as f:
      f.write(json.dumps(match_data, indent=4))
    f.close

  return final_crop


def stitch_two(img_lst, show_images=False, save_file=False, save_directory=None, save_filename=None, write_logs=False):
  """
  Stitches two images together in one round, including automatic cropping of columns.
  (as required) in between rounds of stitching, with final crop of the sky.

  Given input images 0,1 there is 1 round of image stitching:
  First round images: 01 (i.e. all images)

  Arguments:
  img_lst         --  list; a list of 6 images to be stitched together,
                      each of which is a color numpy array inage.
  show_images     --  boolean, whether the images should be plotted using matplotlib upon completion.
  save_file       --  boolean, whether the files should be saved upon completion of the stitching.
  save_directory  --  string; location where the file should be saved to.
  save_filename   --  string; name of the file to be saved, e.g. 'stitch6.jpg'.
  write_logs      --  boolean; whether to write JSON file to log stitching keypoint matches for later analysis.

  Returns:
  final_crop      --  numpy array image, color; the stitched and cropped image result.
  """
  matches_lst = [] # list where number of keypoint matches will be saved
  i = 0 # reset indexes

  # Show original images
  if show_images == True:
    num_cols = 2
    num_rows = 1
    idx = 0
    plt.figure(figsize=[10,10])
    for idx in range(0, len(img_lst)):
      plt.subplot(num_rows, num_cols, idx + 1)
      plt.title('Original Images')
      plt.axis("off")
      plt.imshow(img_lst[idx])

  # First (and only) round of image stitching --> two images in img_lst: 0 and 1
  # Upon stitching, one resulting stitched image: 01, stored in stitched_img
  for i in range(0,len(img_lst)-1,2):
    stitched_img, num_matches = stitch_it(img_lst[i], img_lst[i+1])
    matches_lst.append(num_matches)

  if show_images == True:
    plt.figure(figsize=[10,10])
    plt.title('Stitch 1')
    plt.axis("off")
    plt.imshow(stitched_img)

  # Crop sky and any remaining black columns/rows from first-round stitches
  final_crop = autocrop_col(stitched_img)
  final_crop = autocrop_row(final_crop)
  final_crop = autocrop_sky(final_crop)

  if show_images == True:
    plt.figure(figsize=[10,10])
    plt.title('Final Crop')
    plt.imshow(final_crop)

  if save_file == True:
    # Write file to disk
    cv2.imwrite(os.path.join(save_directory, save_filename), cv2.cvtColor(final_crop, cv2.COLOR_BGR2RGB))

  if write_logs == True:
    # Log matches into JSON file for later analysis
    match_logpath = os.path.join(MAIN_FOLDER,'match_log.json')
    match_data = {}
    match_data['image_id'] = image_id
    match_data['matches'] = matches_lst
    with open(match_logpath, 'a') as f:
      f.write(json.dumps(match_data, indent=4))
    f.close

  return final_crop


def stitching_main(img_lst, show_images=False, save_file=False, save_directory=None, save_id=None, error_log_dir=PROJECT_DIR, write_logs=False):
  """
  Main stitching function, selects whether to use stitch_two, stitch_four, or stitch_six based on
  number of input images.
  If there are errors during stitching, the function logs these to the noted path/file, rather
  than throwing errors during stitching. Errors occur when images cannot be stitched together,
  e.g. if images are too close from GSV camera to building.

  Arguments:
  img_lst         --  list, a list of 2, 4, or 6 images to be stitched together
  show_images     --  boolean, whether the images should be plotted using matplotlib upon completion
  save_file       --  boolean, whether the files should be saved upon completion of the stitching
  save_directory  --  string, the location where the file should be saved to
  save_id         --  string, the file name to save to
  save_filename   --  string, the name of the file to be saved, e.g. 'stitch2.jpg'
  error_log_dir   --  string, the directory where error logs should be saved to
  """
  if not os.path.exists(save_directory):
    os.makedirs(save_directory)

  try:
    if len(img_lst) == 2:
      stitch_two(img_lst, show_images=show_images, save_file=save_file, save_directory=save_directory, save_filename=str(save_id)+'.jpg', write_logs=write_logs)

    elif len(img_lst) == 4:
      stitch_four(img_lst, show_images=show_images, save_file=save_file, save_directory=save_directory, save_filename=str(save_id)+'.jpg', write_logs=write_logs)

    elif len(img_lst) == 6:
      stitch_six(img_lst, show_images=show_images, save_file=save_file, save_directory=save_directory, save_filename=str(save_id)+'.jpg', write_logs=write_logs)

  except Exception as e:
    error_logpath = os.path.join(error_log_dir,'error_log.json')
    error_data = {}
    error_data['image_id'] = image_id
    error_data['error'] = str(e)
    with open(error_logpath, 'a') as f:
      f.write(json.dumps(error_data, indent=4))
    f.close


# For loop to call functions and extract multiple images

In [ ]:
# if folders do not exist, create them
if not os.path.exists(MAIN_FOLDER):
  os.makedirs(MAIN_FOLDER)
if not os.path.exists(STITCH_FOLDER):
  os.makedirs(STITCH_FOLDER)

# loop through rows of dataframe to extract all images
for row in frontlines_df.index:

  # initialize empty lists
  heading_lst= []
  image_lst = []

  # extract paramaters from dataframe and set variables for each iteration of the loop, using parse_dataframe function
  image_id, id_folder, center_lon, center_lat, corner_a_tup, corner_b_tup, perp_angle = parse_dataframe(row, frontlines_df)
  print('image_id: ', image_id)

  # get central camera position from GSV API using get_center_camera function
  camera_lon, camera_lat, img0 = get_center_camera(perp_angle, FOV, center_lon, center_lat, id_folder, radius=RADIUS, offset_dist=6)
  if camera_lon == False:
    continue

  # calculate heading to left and right corner of building using rotation_one_camera function
  heading_left, heading_right = rotation_one_camera(camera_lon, camera_lat, corner_a_tup[0], corner_a_tup[1],
                                                    corner_b_tup[0], corner_b_tup[1], perp_angle, fov=FOV, correction_factor=ROTATION_CORRECTION)

  # create list of headings to send to GSV API using parse_headings function
  heading_lst = parse_headings(perp_angle, heading_left, heading_right)

  # get photos from GSV according to headings in head_lst using get_gsv_photos
  image_lst = get_gsv_photos(ortho_img = img0, save_folder=id_folder, camera_lon=camera_lon, camera_lat=camera_lat,
                             head_lst=heading_lst, pitch_lst=PITCH_LST, fov=FOV, key=KEY, radius=RADIUS)

  # using image_lst, stitch photos together and save to directory
  stitching_main(image_lst, show_images=False, save_file=True, save_directory=STITCH_FOLDER,
                 save_id=image_id, error_log_dir=PROJECT_DIR, write_logs=True)


# Next steps

In [ ]:
'''

NEXT STEPS:

(i) Annotate extracted images using custom Rhino-based tool or other annotation tool.
    --> See folder 03_Annotation_tool in this Repository
    --> If using the trained YOLOv9 model in this Repository, at a minimum, it is suggested to annotate 
    images to run a test set and check performance of the model on the local architectural context.
    --> Images can also be annotated for additional training and validation sets.

(ii) Upon completion of annotations, run deep learning model using extracted images and annotations.
    --> Can use pre-trained YOLOv9 model in this Repository as is (if test set outcomes are satisfactory), 
    and/or with additional training using newly extracted images and annotations.
    --> Another deep learning archiecture could also be used to train/validate a new model.

(iii) Upon completion of the deep learning stage, the post-scripts to run additional NMS and stratify detections into ground-floor and above-ground-floor
    --> See folder 04_Post_scripts_NMS_and_stratify_detections in this Repository.

    
'''